In [1]:
import spacy
from spacy import displacy
from spacy_llm.util import assemble
import spacy


nlp = spacy.load('en_core_web_md')
cache_dir = "./llm_cache"

# actually just do an external config file

nlp = assemble("config.cfg")


/Users/tombedor/development/youbot/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

llm_config = {
    "task": {
        "@llm_tasks": "spacy.REL.v1",
        "labels": ["work for", "founder of", "interested in", "lives in", "associated with", "attendees of", "organizer of"]
    },
    "model": {
        "@llm_models": "spacy.Dolly.v1",
        "name": "dolly-v2-3b"  # For better performance, use 'dolly-v2-12b' instead
    },
    "cache": cache_dir,
    "save_io": True  # or False, depending on your needs
}
nlp.add_pipe("llm", config=llm_config)

In [ ]:

from youbot.store import Store


MESSAGES_COUNT = 10


# 0. Create documents as transcription logs of converstaions
docs = Store().get_archival_messages()[:MESSAGES_COUNT]


In [ ]:

# For NER visualization:
displacy.render(doc, style='ent', jupyter=True)

# For dependency visualization:
displacy.render(doc, style='dep', jupyter=True)